## Установка и импорт зависимостей

In [1]:
!pip install pandas numpy tqdm sklearn CurrencyConverter

In [2]:
import pandas
import re
import numpy
import pickle
from tqdm import tqdm
import helpers
from sklearn import preprocessing
from currency_converter import CurrencyConverter
cc = CurrencyConverter()

## Загрузка данных

In [3]:
df: pandas.DataFrame = pandas.read_csv('real_estate_data.csv', low_memory=False, index_col=0)

### Замена значений из таблицы для лучшего понимания

In [4]:
replace_map = {
}
df = df.replace(replace_map)

### Определяю столбцы без уникальных значений и убираю их

In [5]:
zero_unique_collumns = []
for name, col in df.iteritems():
    print(name + " : " + str(len(col.unique())))
    if(len(col.unique()) <= 1):
        zero_unique_collumns.append(name)

type : 1
sub_type : 12
start_date : 181
end_date : 182
listing_type : 3
tom : 181
building_age : 15
total_floor_count : 13
floor_no : 36
room_count : 37
size : 944
address : 7842
furnished : 1
heating_type : 17
price : 3704
price_currency : 5


In [6]:
print(zero_unique_collumns)
df = df.drop(labels=zero_unique_collumns, axis=1)

['type', 'furnished']


### Разделяю адрес на несколько стобцов для лучшей работы классификатора

In [7]:
def splitAddressCol(val: str):
    return val.split('/')

df = helpers.split_collumn(df, 'address', ['address1', 'address2', 'addres2'], splitAddressCol)

100%|██████████| 403487/403487 [00:00<00:00, 465952.68it/s]


### Разделю столбец с количеством комнат для лучшей работы классификатора

In [8]:
def splitRoomCol(val: str):
    result = re.findall(r"(\d+)\+(\d+)", val)
    if(len(result) == 1):
        return [int(result[0][1]), int(result[0][0]) + int(result[0][1])]
    else:
        return []

df = helpers.split_collumn(df, 'room_count', ['livingroom_count', 'room_count'], splitRoomCol)

100%|██████████| 403487/403487 [00:01<00:00, 310899.14it/s]


### Перевод цены в одну валюту

In [9]:
df = df[df['price_currency'].notna()]

In [10]:
def currencyConvert(row):
    return cc.convert(row['price'], row['price_currency'])

df = helpers.concat_collumns(df, ['price', 'price_currency'], 'price', currencyConvert)

100%|██████████| 402772/402772 [04:23<00:00, 1526.21it/s]


### Сохранение незакодированных даных для удобства

In [15]:
df.to_csv('uncoded_data.csv')

###  Кодирование нечисловых значений

100%|██████████| 16/16 [00:02<00:00,  7.34it/s]


#### Сохранение моделей кодировшиков для разкодирования в будуюшем 

In [12]:
with open('les.pkl', 'wb') as output:
    pickle.dump(les, output)

## Сохранение отфильтрованных данных

In [13]:
df.to_csv('preprocessed_data.csv')

In [14]:
    df

,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,size,heating_type,address1,address2,addres2,livingroom_count,room_count,price
id,,,,,,,,,,,,,,,,
1,7,93,30,2,30,0,4,11,90.0,0,78,249,2446,1.0,3.0,4.553793e+02
2,0,127,181,1,14,0,4,12,43.0,0,78,249,2446,0.0,1.0,6.375311e+04
3,0,61,90,1,30,0,0,32,NaN,0,66,488,3253,1.0,3.0,2.016680e+04
4,7,152,32,1,30,7,4,12,450.0,0,78,88,2746,1.0,7.0,4.228522e+06
5,7,93,30,1,30,0,4,11,90.0,0,78,249,2446,1.0,3.0,1.886572e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403483,0,160,181,2,162,14,12,34,NaN,16,78,394,96,NaN,NaN,1.951626e+02
403484,0,33,181,1,139,14,12,34,NaN,16,61,4,1012,1.0,3.0,1.561301e+04
403485,0,76,181,1,97,14,12,34,NaN,16,6,22,3331,1.0,2.0,4.800000e+04
